In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

rootDirectory = '/content/drive/My Drive/ml/'
trainData = rootDirectory + 'data.hdf5'
testData = rootDirectory + 'test.hdf5'
bestModel = rootDirectory + 'best_model.hdf5'
tfModel = rootDirectory + 'tfModel'
tfliteModel = rootDirectory + 'model.tflite'

In [ ]:
# import modules
import h5py
import numpy as np
from tensorflow import lite
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# load and normalize train and test input data
f = h5py.File(trainData, 'r')
trainX = f['trainX'][:]/255
f.close()

f = h5py.File(testData, 'r')
testX = f['trainX'][:]/255
f.close()

# create labels for train and test prediction data
# input order: 1-400 (ipod), 401-1180 (keys), 1181-1780 (ipod), 1781-2000 (keys), 2001-3000 (nothing)

# predition labels:
# [1, 0, 0] - ipod
# [0, 1, 0] - keys
# [0, 0, 1] - nothing

trainY = np.array([[1, 0, 0]]*400)
trainY = np.concatenate((trainY, np.array([[0, 1, 0]]*780)))
trainY = np.concatenate((trainY, np.array([[1, 0, 0]]*600)))
trainY = np.concatenate((trainY, np.array([[0, 1, 0]]*220)))
trainY = np.concatenate((trainY, np.array([[0, 0, 1]]*1000)))

testY = np.array([[1, 0, 0]]*100)
testY = np.concatenate((testY, np.array([[0, 1, 0]]*195)))
testY = np.concatenate((testY, np.array([[1, 0, 0]]*150)))
testY = np.concatenate((testY, np.array([[0, 1, 0]]*55)))
testY = np.concatenate((testY, np.array([[0, 0, 1]]*250)))

In [ ]:
# create AlexNet model
def model(input_shape):
    X_input = Input(input_shape)
    
    X = Conv2D(96, (11, 11), strides = (4, 4), name = "conv1")(X_input)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu', name = 'relu1')(X)
    X = MaxPooling2D((3, 3), strides = (2, 2), name = 'max_pool1')(X)

    X = Conv2D(256, (5, 5), padding='same', name = "conv2")(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu', name = 'relu2')(X)
    X = MaxPooling2D((3, 3), strides = (2, 2), name = 'max_pool2')(X)

    X = Conv2D(384, (3, 3), padding='same', name = "conv3")(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu', name = 'relu3')(X)

    X = Conv2D(384, (3, 3), padding='same', name = "conv4")(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu', name = 'relu4')(X)

    X = Conv2D(256, (3, 3), padding='same', name = "conv5")(X)
    X = BatchNormalization(axis = 3, name = 'bn5')(X)
    X = Activation('relu', name = 'relu5')(X)
    X = MaxPooling2D((3, 3), strides = (2, 2), name = 'max_pool3')(X)

    X = Flatten()(X)
    X = Dense(4096, name='fc1')(X)
    X = BatchNormalization(axis = 1, name = 'bn6')(X)
    X = Activation('relu', name = 'relu6')(X)

    X = Dense(4096, name='fc2')(X)
    X = BatchNormalization(axis = 1, name = 'bn7')(X)
    X = Activation('relu', name = 'relu7')(X)

    X = Dense(3, activation='softmax', name='fc3')(X)

    model = Model(inputs = X_input, outputs = X, name='AlexNet')

    return model

alexNet = model(trainX.shape[1:])

In [ ]:
# load best trained model
alexNet = load_model(bestModel)

In [ ]:
# set hyperparameters and train model
adam = optimizers.Adam(learning_rate=0.000001)
alexNet.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint(bestModel, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq='epoch')

datagen = ImageDataGenerator(rotation_range=360,
                             width_shift_range=0.05,
                             height_shift_range=0.05,
                             brightness_range=[0.8, 1.2],
                             shear_range=1.0,
                             zoom_range=0.1,
                             channel_shift_range=0.1,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rescale=1.0/255.0)

alexNet.fit(datagen.flow(trainX, trainY, batch_size=64), steps_per_epoch=len(trainX) / 64, epochs=1000, callbacks=[checkpoint])

In [ ]:
# evaluate best trained model
alexNet = load_model(bestModel)
preds = alexNet.evaluate(testX, testY)
print()
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

In [ ]:
# save model in tflite format

# save model in tf format
alexNet.save(tfModel, include_optimizer=False, save_format='tf')

# convert model to tflite format
converter = lite.TFLiteConverter.from_saved_model(tfModel)
tflite_model = converter.convert()

# save tflite model
with open(tfliteModel, 'wb') as f:
  f.write(tflite_model)
